In [0]:
!pip install -q tensorflow-gpu==2.0.0

In [0]:
import tensorflow_datasets as tfds
import tensorflow as tf
import multiprocessing

In [0]:
def create_model():
  input_layer = tf.keras.layers.Input(shape=(224, 224, 3))
  base_model = tf.keras.applications.MobileNetV2(input_tensor=input_layer,
                                                 weights='imagenet',
                                                 include_top=False)
  base_model.trainable = False
  x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
  x = tf.keras.layers.Dense(2, activation='softmax')(x)
  
  model = tf.keras.models.Model(inputs=input_layer, outputs=x)
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', 
              metrics=['acc'])
  return model

In [0]:
dataset_name = 'cats_vs_dogs'
dataset, info = tfds.load(name=dataset_name, split=tfds.Split.TRAIN, with_info=True)

In [0]:
print(info.version)
info.version

# Naively training with tf.data

Let us run through the code in a normal scenario. We will not use any of the new concepts of parallelization we have learnt in this module.

In [0]:
def preprocess(features):
  image = features['image']
  image = tf.image.resize(image, (224, 224))
  image = image / 255.0
  return image, features['label']

In [0]:
train_dataset = dataset.map(preprocess).batch(32)

In [0]:
model = create_model()
model.fit(train_dataset, epochs=5)

# Exercise

This exercise is about parallelizing various stages of Extract, Transform and Load processes. In this exercise, you will be tasked with following tasks:   

1.   Parallelize extraction of stored TFRecords of cats_vs_dogs dataset using interleave operation.
2.   Parallelize transformation during preprocessing of raw dataset using map operation.
3.   Cache the processed dataset in memory using cache operation for faster retrieval.
4.   Parallelize the loading of cached dataset during training cycle using prefetch operation.



In [0]:
file_pattern = f'/root/tensorflow_datasets/{dataset_name}/{info.version}/{dataset_name}-train.tfrecord*'
files = tf.data.Dataset.list_files(file_pattern)

## Parallelize Extraction


In [0]:
# Parallelize extraction of stored TFRecords of cats_vs_dogs dataset using interleave operation
# with cycle length = 4 and number of parallel calls set to AUTOTUNE.
train_dataset = files.interleave(
    tf.data.TFRecordDataset, cycle_length=4,
    num_parallel_calls=tf.data.experimental.AUTOTUNE)

## Parallelize Transformation

In [0]:
# Parallelize transformation of extracted dataset using map operation
def read_tfrecord(serialized_example):
  # Describe the feature description dictionary for the supervised keys
  feature_description = {
      'image': tf.io.FixedLenFeature((), tf.string, ""),
      'label': tf.io.FixedLenFeature((), tf.int64, -1),
  }
  # Parse the example image and decode it
  example = tf.io.parse_single_example(serialized_example, feature_description)
  image = tf.io.decode_jpeg(example['image'], channels=3)
  image = tf.cast(image, tf.float32)
  # Normalize the image
  image = image / 255.
  # Resize the image
  image = tf.image.resize(image, (224, 224))
  return image, example['label']

In [0]:
# Get the number of CPU cores. 
cores = multiprocessing.cpu_count()
print(cores)

# Apply the map transformation with number of parallel calls set to number of cores
train_dataset = train_dataset.map(read_tfrecord, num_parallel_calls=cores)

## Cache the dataset

In [0]:
# Cache the dataset in-memory
train_dataset = train_dataset.cache()

## Parallelize Loading

In [0]:
# Shuffle and batch the dataset
train_dataset = train_dataset.shuffle(1024).batch(32)
# Parallelize the loading by prefetching the dataset and setting buffer size to AUTOTUNE
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [0]:
model = create_model()
model.fit(train_dataset, epochs=1)